<a href="https://colab.research.google.com/github/Yash-Kavaiya/AI-Accelerate-Retail-Agent-Teams/blob/main/product_image_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install git+https://github.com/facebookresearch/ImageBind.git
!pip install elasticsearch pandas pillow requests


Looking in indexes: https://download.pytorch.org/whl/cu118
  Cloning https://github.com/facebookresearch/ImageBind.git to /tmp/pip-req-build-th8ylt9o
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/ImageBind.git /tmp/pip-req-build-th8ylt9o
  Resolved https://github.com/facebookresearch/ImageBind.git to commit cfee5753e8e0b7a9df8aeb19c493e8facd81a5f2
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/pytorchvideo.git (to revision 6cdc929315aab1b5674b6dcf73b16ec99147735f) to /tmp/pip-install-yjdw7e65/pytorchvideo_f81c7aeb30a348768f150bd90f89a8d8
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorchvideo.git /tmp/pip-install-yjdw7e65/pytorchvideo_f81c7aeb30a348768f150bd90f89a8d8
  Running command git rev-parse -q --verify 'sha^6cdc929315aab1b5674b6dcf73b16ec99147735f'
  Running command git fetch -q https://github.com/facebookresearch/pytorchvideo.git 6cdc929315aab1b5

In [ ]:
import torch
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from imagebind import data
from imagebind.models import imagebind_model
from imagebind.models.imagebind_model import ModalityType
from PIL import Image
import requests
from io import BytesIO
import numpy as np
from tqdm import tqdm
import os

# Configuration
ES_CLOUD_ID = "my-elasticsearch-project-e0ae1f.es.us-central1.gcp.elastic.cloud:443"
ES_API_KEY = "cTBuSkVKb0JDNDR2WWhEajVsWjQ6UkhPNEtaZkJTa2VoWTEtVmxtbW8wUQ=="
INDEX_NAME = "imagebind-embeddings"
BATCH_SIZE = 16  # Reduce if you hit OOM errors
MAX_RETRIES = 3

# Check GPU availability
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Initialize Elasticsearch client
client = Elasticsearch(
    f"https://{ES_CLOUD_ID}",
    api_key=ES_API_KEY,
    request_timeout=120
)

# Test connection
if client.ping():
    print("Successfully connected to Elasticsearch!")
else:
    raise Exception("Could not connect to Elasticsearch")

# Load ImageBind model
print("Loading ImageBind model...")
model = imagebind_model.imagebind_huge(pretrained=True)
model.eval()
model.to(device)
print("Model loaded successfully!")

def create_index():
    mapping = {
        "mappings": {
            "properties": {
                "filename": {"type": "keyword"},
                "image_url": {"type": "text"},
                "gender": {"type": "keyword"},
                "masterCategory": {"type": "keyword"},
                "subCategory": {"type": "keyword"},
                "articleType": {"type": "keyword"},
                "baseColour": {"type": "keyword"},
                "season": {"type": "keyword"},
                "year": {"type": "integer"},
                "usage": {"type": "keyword"},
                "productDisplayName": {"type": "text"},
                "image_embedding": {
                    "type": "dense_vector",
                    "dims": 1024,
                    "index": True,
                    "similarity": "cosine"
                }
            }
        }
    }
    if client.indices.exists(index=INDEX_NAME):
        client.indices.delete(index=INDEX_NAME)
        print(f"Deleted existing index: {INDEX_NAME}")
    client.indices.create(index=INDEX_NAME, body=mapping)
    print(f"Created index: {INDEX_NAME}")

def load_images_csv(path):
    try:
        df = pd.read_csv(path, delimiter=',', on_bad_lines='skip')
        if 'filename' in df.columns and 'link' in df.columns and not df.empty:
            return df
    except Exception:
        pass
    # Recovery for pair-per-row format
    fixed_rows = []
    with open(path, 'r', encoding='utf-8') as f:
        lines = [line.strip() for line in f if line.strip()]
        # Ignore header if present
        # If lines[0] doesn't end with ".jpg" or look like an image url, skip as header
        start_idx = 0
        if not (lines[0].endswith(".jpg") or lines[0].startswith("http")):
            start_idx = 1
        for i in range(start_idx, len(lines)-1, 2):
            fname = lines[i]
            link = lines[i+1]
            fixed_rows.append([fname, link])
    df = pd.DataFrame(fixed_rows, columns=['filename', 'link'])
    return df

def load_styles_csv(path):
    return pd.read_csv(path, delimiter=',', on_bad_lines='skip')

def load_data():
    print("Loading CSV files...")
    images_df = load_images_csv('/content/images.csv')
    styles_df = load_styles_csv('/content/styles.csv')
    images_df['id'] = images_df['filename'].str.replace('.jpg', '').astype(int)
    merged_df = images_df.merge(styles_df, on='id', how='inner')
    print(f"Loaded {len(merged_df)} records")
    return merged_df

def download_image(url, retries=MAX_RETRIES):
    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=15)
            if response.status_code == 200:
                img = Image.open(BytesIO(response.content))
                return img.convert('RGB')
        except Exception as e:
            if attempt == retries - 1:
                print(f"Failed after {retries} attempts: {url[:50]}...")
            continue
    return None

def generate_embeddings_batch(image_urls, batch_indices):
    valid_images = []
    valid_indices = []
    temp_paths = []
    for idx, url in enumerate(image_urls):
        img = download_image(url)
        if img is not None:
            temp_path = f"/tmp/temp_img_{batch_indices[idx]}.jpg"
            img.save(temp_path)
            valid_images.append(temp_path)
            valid_indices.append(idx)
            temp_paths.append(temp_path)
    if not valid_images:
        return None, []
    try:
        inputs = {
            ModalityType.VISION: data.load_and_transform_vision_data(valid_images, device)
        }
        with torch.no_grad():
            embeddings = model(inputs)
            vision_embeddings = embeddings[ModalityType.VISION]
        embeddings_np = vision_embeddings.cpu().numpy()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        return embeddings_np, valid_indices
    except Exception as e:
        print(f"Error generating embeddings: {e}")
        return None, []
    finally:
        # Clean up temp files
        for path in temp_paths:
            try:
                if os.path.exists(path):
                    os.remove(path)
            except:
                pass

def index_documents(df):
    print("Generating embeddings and indexing to Elasticsearch...")
    actions = []
    successful = 0
    failed = 0
    total_batches = (len(df) + BATCH_SIZE - 1) // BATCH_SIZE
    for batch_idx in tqdm(range(0, len(df), BATCH_SIZE), total=total_batches):
        batch = df.iloc[batch_idx:batch_idx+BATCH_SIZE]
        batch_indices = list(range(batch_idx, min(batch_idx+BATCH_SIZE, len(df))))
        embeddings, valid_indices = generate_embeddings_batch(batch['link'].tolist(), batch_indices)
        if embeddings is None:
            failed += len(batch)
            continue
        for idx, embedding in zip(valid_indices, embeddings):
            row = batch.iloc[idx]
            doc = {
                "_index": INDEX_NAME,
                "_id": str(row['id']),
                "_source": {
                    "filename": row['filename'],
                    "image_url": row['link'],
                    "gender": row.get('gender', ''),
                    "masterCategory": row.get('masterCategory', ''),
                    "subCategory": row.get('subCategory', ''),
                    "articleType": row.get('articleType', ''),
                    "baseColour": row.get('baseColour', ''),
                    "season": row.get('season', ''),
                    "year": int(row.get('year', 0)) if pd.notna(row.get('year')) else 0,
                    "usage": row.get('usage', ''),
                    "productDisplayName": row.get('productDisplayName', ''),
                    "image_embedding": embedding.tolist()
                }
            }
            actions.append(doc)
            successful += 1
        if len(actions) >= 50:
            try:
                success_count, errors = bulk(client, actions, raise_on_error=False)
                if errors:
                    print(f"Bulk index errors: {len(errors)}")
                actions = []
            except Exception as e:
                print(f"Error during bulk indexing: {e}")
                failed += len(actions)
                actions = []
    if actions:
        try:
            bulk(client, actions, raise_on_error=False)
        except Exception as e:
            print(f"Error indexing final batch: {e}")
    print(f"\n{'='*50}")
    print(f"Indexing completed!")
    print(f"Successfully indexed: {successful}")
    print(f"Failed: {failed}")
    print(f"{'='*50}")

def search_similar_images(query_text=None, query_image_path=None, top_k=10, filters=None):
    if query_text:
        inputs = {
            ModalityType.TEXT: data.load_and_transform_text([query_text], device)
        }
    elif query_image_path:
        inputs = {
            ModalityType.VISION: data.load_and_transform_vision_data([query_image_path], device)
        }
    else:
        raise ValueError("Provide either query_text or query_image_path")
    with torch.no_grad():
        embeddings = model(inputs)
        if query_text:
            query_embedding = embeddings[ModalityType.TEXT][0].cpu().numpy()
        else:
            query_embedding = embeddings[ModalityType.VISION][0].cpu().numpy()
    search_query = {
        "knn": {
            "field": "image_embedding",
            "query_vector": query_embedding.tolist(),
            "k": top_k,
            "num_candidates": 100
        },
        "_source": ["filename", "image_url", "productDisplayName", "baseColour", "gender", "articleType"]
    }
    if filters:
        filter_clauses = []
        for field, value in filters.items():
            filter_clauses.append({"term": {field: value}})
        search_query["knn"]["filter"] = {"bool": {"must": filter_clauses}}
    results = client.search(index=INDEX_NAME, body=search_query)
    print(f"\n{'='*60}")
    print(f"Search Query: {query_text if query_text else query_image_path}")
    if filters:
        print(f"Filters: {filters}")
    print(f"{'='*60}\n")
    print(f"Top {len(results['hits']['hits'])} similar images:\n")
    for i, hit in enumerate(results['hits']['hits'], 1):
        source = hit['_source']
        print(f"{i}. Score: {hit['_score']:.4f}")
        print(f"   Product: {source.get('productDisplayName', 'N/A')}")
        print(f"   Type: {source.get('articleType', 'N/A')} | Colour: {source.get('baseColour', 'N/A')}")
        print(f"   URL: {source['image_url'][:70]}...")
        print()
    return results

def main():
    create_index()
    df = load_data()
    # df = df.head(100)  # Uncomment for quick test
    index_documents(df)
    client.indices.refresh(index=INDEX_NAME)
    print("\n" + "="*60)
    print("EXAMPLE SEARCHES")
    print("="*60)
    print("\n1. Text Search: 'blue shirt for men'")
    search_similar_images(query_text="blue shirt for men", top_k=5)
    print("\n2. Filtered Search: 'casual watch' for Women")
    search_similar_images(query_text="casual watch", top_k=5, filters={"gender": "Women"})
    stats = client.indices.stats(index=INDEX_NAME)
    doc_count = stats['indices'][INDEX_NAME]['primaries']['docs']['count']
    print(f"\n{'='*60}")
    print(f"Total documents indexed: {doc_count}")
    print(f"{'='*60}")

if __name__ == "__main__":
    main()


Using device: cuda:0
GPU: Tesla T4
GPU Memory: 15.83 GB
Successfully connected to Elasticsearch!
Loading ImageBind model...
Model loaded successfully!
Deleted existing index: imagebind-embeddings
Created index: imagebind-embeddings
Loading CSV files...
Loaded 44424 records
Generating embeddings and indexing to Elasticsearch...


  0%|          | 8/2777 [02:04<11:53:58, 15.47s/it]

Bulk index errors: 2


  1%|          | 20/2777 [05:17<14:03:29, 18.36s/it]

Bulk index errors: 2


...

 31%|███       | 856/2777 [3:57:48<8:36:37, 16.14s/it]

Bulk index errors: 1


 31%|███       | 860/2777 [3:58:57<9:00:35, 16.92s/it]

Bulk index errors: 1


 31%|███       | 861/2777 [3:59:13<8:59:57, 16.91s/it]